***Visualizing PCA with Leaf Dataset***
================================

In this script we will apply PCA on leaf images and try to get a feel for the distribution of leaf images using visualizations that (hopefully) clarify different aspects about how to interpret PCA results.

We will then continue to see if the PCA features are informative in terms of classifying leafs and determine how many of those we need.

In [ ]:
import os
import cv2

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from sklearn import model_selection
from sklearn import decomposition
from sklearn import linear_model
from sklearn import ensemble
from sklearn import neighbors
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KernelDensity
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score
from sklearn import svm

from skimage.transform import rescale
from scipy import ndimage as ndi

matplotlib.style.use('fivethirtyeight')

**For the sake of the script not being too cluttered, I commented out all intermediate plots in the data loading and preparation phases**

anyone who is interested is welcome to fork and uncomment to see what is going on.

(the main assumption of this pre-processing stage is that the absolute sizes of the leafs matter, and not just their shape. i.e. leafs with different sizes are most definitely different types of leafs. not sure if it's actually important, but just in case)

In [ ]:
#%% load the data
dataDir = '../input/'
trainData = pd.read_csv(dataDir + 'train.csv')
classEncoder = LabelEncoder()
trainLabels = classEncoder.fit_transform(trainData.ix[:,'species'])
trainIDs = np.array(trainData.ix[:,'id'])

dat = np.array(trainData.ix[:,2:194])




In [ ]:
#Extract HuMomments - Binary shape feature
from sklearn import preprocessing

numImages = 1584
n=100
humomments = np.zeros([numImages,7])
i=0
for i in range (numImages):
    imageFilename = dataDir + 'images/' + str(i+1) + '.jpg'
    img = mpimg.imread(imageFilename)           
    humomments[i-1,:] = np.transpose(cv2.HuMoments(cv2.moments(img)))
    i = i+1
    
#print(humomments)
normmomments = preprocessing.normalize(humomments, axis=1 ,norm='l2')
scaledmomments = preprocessing.scale(humomments,axis=1)

In [ ]:
print (normmomments.shape)
print(dat.shape)

In [ ]:
#%% define GaussianModel class

class GaussianModel:
    def __init__(self, X, numBasisFunctions=10, objectPixels=None):
        '''
        inputs: 
            X                    - numSamples x numDimentions matrix
            numBasisFunctions       - number of basis function to use
            objectPixels (optional) - an binnary mask image used for presentation
                                      will be used as Im[objectPixels] = dataSample
                                      must satisfy objectPixels.ravel().sum() = X.shape[1]
        '''
        
        self.numBasisFunctions = numBasisFunctions        
        if objectPixels == None:
            self.objectPixels = np.ones((1,X.shape[1]),dtype=np.bool)
        else:
            self.objectPixels = objectPixels
        assert(self.objectPixels.ravel().sum() == X.shape[1])

        PCAModel = decomposition.PCA(n_components=numBasisFunctions, whiten=True)
        self.dataRepresentation = PCAModel.fit_transform(X)
        self.PCAModel = PCAModel

    def RepresentUsingModel(self, X):
        return self.PCAModel.transform(X)


In [ ]:
# train the Gaussian Model 

#sampleDim = np.shape(scaledDownImages)[0]*np.shape(scaledDownImages)[1]
#X = scaledDownImages.reshape(sampleDim,-1).T
#
#objectPixelsMask = np.ones((np.shape(scaledDownImages)[0],np.shape(scaledDownImages)[1]))==1
#leaf_PCAModel = GaussianModel(X, numBasisFunctions=100, objectPixels=objectPixelsMask)
X=trainData.filter(items=trainData.columns[2:])
leaf_PCAModel = GaussianModel(X, numBasisFunctions=192, )

In [ ]:
matplotlib.rcParams['font.size'] = 5
matplotlib.rcParams['figure.figsize'] = (8,5)

X_PCA = leaf_PCAModel.RepresentUsingModel(X)
X_PCA

X_PCA_train = X_PCA     #   [trainIDs-1,:]
y_train = trainLabels

tSNE_PCAModel = TSNE(n_components=2, random_state=0)
X_PCA_train_tSNE = tSNE_PCAModel.fit_transform(X_PCA_train) 

plt.figure()
plt.subplot(1,2,1); plt.scatter(X_PCA_train[:,0],X_PCA_train[:,1],c=y_train,cmap='Paired',s=10,alpha=0.95)
plt.title('PCA representation'); plt.xlabel('PC1 coeff'); plt.ylabel('PC2 coeff')
plt.subplot(1,2,2); plt.scatter(X_PCA_train_tSNE[:,0],X_PCA_train_tSNE[:,1],c=y_train,cmap='Paired',s=10,alpha=0.95)
plt.title('t-SNE representation'); plt.xlabel('t-SNE axis1'); plt.ylabel('t-SNE axis2')

We can see that nearby points usually have similar color and this means they have similar leaf label. This makes us confident that we can achieve at least some classification accuracy from these PCA features.


----------


Now, let's see what is the **classification accuracy** using this PCA representation if we use **different amount of PCA coefficients** for several different types of classifiers.

In [ ]:
#%% plot CV classification accuracy as function of num components used for 3 very different type of classifiers
matplotlib.rcParams['font.size'] = 7
matplotlib.rcParams['figure.figsize'] = (8,7)

numPCsToUse = [1,2,4,8,16,32,64,96,128,160,192]

logReg = linear_model.LogisticRegression(C=10.0)
kNN = neighbors.KNeighborsClassifier(n_neighbors=7)
RF = ensemble.RandomForestClassifier(n_estimators=100)
Svm = svm.SVC(gamma=0.1, C=100.)

ClsDict={
    'logReg': {'cls': logReg, 'Acc':[], 'meanAcc':[], 'AccStd':[]},
    'kNN': {'cls': kNN, 'Acc':[], 'meanAcc':[], 'AccStd':[]},
    'RF': {'cls': RF, 'Acc':[], 'meanAcc':[], 'AccStd':[]},
    'Svm': {'cls': Svm, 'Acc':[], 'meanAcc':[], 'AccStd':[]},
}
logRegMeanAccuracy = []; kNN_MeanAccuracy = []; RF_MeanAccuracy = []; Svm_MeanAccuracy = []
logRegAccuracyStd  = []; kNN_AccuracyStd  = []; RF_AccuracyStd  = []; Svm_AccuracyStd  = []

for numPCs in numPCsToUse:
    stratifiedCV = model_selection.StratifiedKFold(n_splits=5, random_state=1)
    for cls in ClsDict:
        ClsDict[cls]['Acc']=[]
    for trainInds, validInds in stratifiedCV.split(X_PCA_train, y_train):
        X_train_cv = X_PCA_train[trainInds,:numPCs]
        X_valid_cv = X_PCA_train[validInds,:numPCs]

        y_train_cv = y_train[trainInds]
        y_valid_cv = y_train[validInds]

        for cls in ClsDict:
            clsObj=ClsDict[cls]['cls']
            clsObj.fit(X_train_cv, y_train_cv)
            ClsDict[cls]['Acc'].append(accuracy_score(y_valid_cv, clsObj.predict(X_valid_cv)))
        
    for cls in ClsDict:
        clsObj=ClsDict[cls]['cls']
        clsObj.fit(X_train_cv, y_train_cv)
        ClsDict[cls]['meanAcc'].append(np.array(ClsDict[cls]['Acc']).mean())
        ClsDict[cls]['AccStd'].append(np.array(ClsDict[cls]['Acc']).std())
        
plt.figure()
for cls in ClsDict:
    plt.errorbar(x=numPCsToUse, y=ClsDict[cls]['meanAcc'], yerr=ClsDict[cls]['AccStd'])
plt.xlim(min(numPCsToUse)-1,max(numPCsToUse)+1); plt.legend(ClsDict.keys(),loc=2)
plt.xlabel('num PCA components'); plt.ylabel('validation accuracy'); plt.title('accuracy as function of num PCs')

Overall, it's evident that all classifiers achieve approximately similar performance.

But it's interesting to note the somewhat different behavior of these different classifiers as a function of number of components used. 

For example, the nearest neighbor classifier flattens out early and does not benefit from additional components beyond 8, whereas the logistic regression classifier continues to increase it's performance up to around 32 components.
The Random Forest classifier is consistently the best performing but it also flattens out at around 32 components.

In [ ]:
ClsDict